# Class 과제

`Class` 를 연습할 겸 평소에 관심있었던 프로젝트를 실행해 보려고 합니다.


## SOLID 원칙

`SOLID` 원칙이란 다음 원칙을 포함한다.
[이 포스팅](https://dev-momo.tistory.com/entry/SOLID-%EC%9B%90%EC%B9%99) 을 참고했습니다.

- Single Responsibility Principle (단일 책임 원칙) : 하나의 클래스는 하나의 기능만을 가지고 있어야 한다.
- Open-Closed Principle (개방-폐쇄 원칙) : 기존 코드를 변경하지 않고 기능을 수정하거나 추가할 수 있도록 설계해야 한다. (인터페이스 형식을 사용)
- Liskov Substitution Principle(리스코프 치환 원칙) : 자식 클래스의 인스턴스는 부모 클래스의 인스턴스의 모든 요소를 포함해야 한다.
- Interface Segregation Principle(인터페이스 분리 원칙) : 자신이 사용하지 않는 기능에는 영향을 받지 말아야 한다.
- Dependency Inversion Principle(의존 역전 원칙) : 의존관계에서는 추상 클래스 (변하기 어려운 클래스) 의 영향을 받아야 한다.

## 개요
딥러닝 모델을 학습할 때 학습 결과를 출력하는 출력기를 만들어 보려고 한다.

이 프로젝트는 다음과 같은 클래스를 가지고 있다.

- `HistoryPrinter` : 제시된 클래스를 모두 실행하는 클래스
- `BasicPrinter` : 기본 클래스 (인터페이스 클래스). 아래의 모든 클래스는 이 클래스를 상속받는다.
- `AccuracyPrinter` : 정확도를 출력하는 클래스
- `LossPrinter` : 손실값을 출력하는 클래스
- `EpochPrinter` : 현재 Epoch와 전체 Epoch을 출력하는 클래스
- `EpochWithPercentagePrinter` : 현재 Epoch와 진행률을 함께 출력하는 클래스
- `BatchPrinter` : 현재 데이터 정보와 전체 데이터셋의 길이를 출력하는 클래스
- `BatchWithPercentagePrinter` : 현재 데이터 정보와 전체 데이터셋의 길이, 진행률까지 함께 출력하는 클래스

In [329]:
# 기본이 되는 클래스 생성 : OCP원칙
class BasicPrinter:
    def __init__(self, isTrain = True):
        self.isTrain = isTrain
    
    def __repr__(self): # 매직 메소드들을 사용해 보겠습니다.
        return f"HistroyPrinter()"

In [330]:
# EpochPrinter 생성
class EpochPrinter(BasicPrinter): # BasicPrinter를 상속 받아서 생성
    
    def __init__(self, totalEpoch, isTrain = True): # 부모 메소드를 오버라이딩
        super().__init__(isTrain)
        self.nowEpoch = None
        self.totalEpoch = totalEpoch
    
    def __repr__(self):
        return "EpochPrinter()" # 기본 클래스를 오버라이딩해서 덮어 씀
    
    
    def show(self, **kwargs): # 로그를 표시합니다. 추후 확장을 고려하여 keyword Arguments 속성을 사용하여 인자를 받겠습니다.
        self.nowEpoch = kwargs["nowEpoch"]
        if self.isTrain:    
            return f"Epoch : [{self.nowEpoch} / {self.totalEpoch}]"
        else:
            return f"Epoch {self.nowEpoch} Test"
        

In [331]:
# EpochWithPercentage 생성
class EpochWithPercentagePrinter(BasicPrinter):
    
    def __init__(self, totalEpoch, isTrain = True):
        super().__init__(isTrain)
        self.nowEpoch = None
        self.totalEpoch = totalEpoch
        
    def __repr__(self):
        return "EpochWithPercentagePrinter()"
    
    
    def show(self, **kwargs):
        self.nowEpoch = kwargs["nowEpoch"]
        if self.isTrain:
            return f"Epoch : [{self.nowEpoch} / {self.totalEpoch}]({self.nowEpoch / self.totalEpoch * 100:0.1f}%)"
        else:
            return f"Epoch {self.nowEpoch}({self.nowEpoch / self.totalEpoch * 100:0.1f}%) Test"
        

In [332]:
# AccuracyPrinter 생성
class AccuracyPrinter(BasicPrinter):
    
    def __init__(self, isTrain = True):
        super().__init__(isTrain)
        
    def __repr__(self):
        return "AccuracyPrinter()" 
    
    def show(self, **kwargs):
        
        output = kwargs["output"] # keyword argument 딕셔너리 안의 속성을 꺼내서 쓸 수 있습니다.
        label = kwargs["label"]
        
        # 정확도 계산
        correct = 0 
        prediction = output.max(1, keepdim=True)[1]
        correct += prediction.eq(label.view_as(prediction)).sum().item()
        accuracy = 100. * correct / len(prediction)
        
        return f"Accuracy : [{accuracy:.3f}%]"
        

In [333]:
# LossPrinter 생성
class LossPrinter(BasicPrinter):
    
    def __init__(self, lossFunc, isTrain = True):
        super().__init__(isTrain)
        self.lossFunc = lossFunc
        
    def __repr__(self):
        return f"LossPrinter(lossFunc = {self.lossFunc})" # 현재 lossFunc 파라미터를 같이 표시했습니다.
    
    def show(self, **kwargs):
        output = kwargs["output"]
        label = kwargs["label"]
        
        loss = self.lossFunc(output, label)
        return f"Loss : [{loss.item():.6f}]"
        
    

In [334]:
# BatchPrinter 생성
class BatchPrinter(BasicPrinter):
    
    def __init__(self, isTrain = True):
        super().__init__(isTrain)
        
    def __repr__(self):
        return f"BatchPrinter()"
    
    def show(self, **kwargs): # 전체 데이터의 길이와 현재 데이터의 번호를 받아서 표시합니다.
        idx = kwargs["index"]
        total = kwargs["totalLen"]
    
        return f"Proceeding.. [{idx} / {total}]"
    

In [335]:
# BatchWithPercentagePrinter 생성
class BatchWithPercentagePrinter(BasicPrinter):
    
    def __init__(self, isTrain = True):
        super().__init__(isTrain)
        
    def __repr__(self):
        return f"BatchWithPercentagePrinter()"
    
    def show(self, **kwargs):
        idx = kwargs["index"]
        total = kwargs["totalLen"]
    
        return f"Proceeding.. [{idx} / {total}]({100. * idx / total:.3f}%)"
    

In [336]:
# HistoryPrinter 생성 -> 전체 학습률을 출력하는 클래스입니다.
class HistoryLogger():
    def __init__(self, printerList, isTrain=True): 
        self.printerList = printerList
        
        if not isTrain: # 만약 생성시에 evaluate 전용으로 생성했다면 내부 클래스들의 isTrain속성을 False 로 설정합니다.
            self.setTrain(isTrain)
            
    def __repr__(self):
        return "HistoryPrinter()" 
    
    
    def allStackedPrinter(self): # 현재 연결되어있는 전체 프린터의 목록을 출력하는 메소드
        for i, printer in enumerate(self.printerList):
            print(f"Printer #{i} : {printer}")
    

    def setTrain(self, isTrain): # 모델의 isTrain설정을 변경합니다.
        self.isTrain = isTrain
        
        for printer in self.printerList: # printerList에 있는 모든 리스트의 속성을 변경합니다.
            printer.isTrain = isTrain


    def show(self,**kwargs):
        for printer in self.printerList:
            if printer == self.printerList[-1]: # 마지막 출력기이면 줄을 분리합니다.
                print(printer.show(**kwargs), end='\n')
            else:
                print(printer.show(**kwargs), end='\t') # 마지막 출력기가 아니면 탭 간격으로 띄어씁니다.
        

## 테스트 코드

파이토치를 임포트해서 MNIST 데이터를 훈련시켜 보고 학습률을 출력해 봅니다.

코드는 `파이썬 딥러닝 파이토치` 책의 튜토리얼 코드를 참고했습니다.

In [337]:
# 모듈 임포트
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets

DEVICE = torch.device('cpu')
BATCH_SIZE = 32
EPOCHS = 3

In [338]:
# MNIST 데이터 다운로드
train_dataset = datasets.MNIST(root = '../data/MNIST', train= True, transform = transforms.ToTensor())
test_dataset = datasets.MNIST(root = '../data/MNIST', train= False, transform = transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [339]:
# 모델 정의
class Net(nn.Module): 
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28 * 28, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)
        
    def forward(self, X):
        x = X.view(-1, 28*28)
        x = self.fc1(x)
        x = F.sigmoid(x)
        x = self.fc2(x)
        x = F.sigmoid(x)
        x = self.fc3(x)
        x = F.log_softmax(x, dim = 1)
        return x

In [340]:
# 모델과 최적화, 손실함수 생성
model = Net().to(DEVICE)
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)
lossFunc = nn.CrossEntropyLoss()

## 위에서 생성한 클래스를 통해 Logger 생성

In [341]:
# 1. 로그를 찍어보고 싶은 항목들을 배열에 담습니다.
loggers = [
    EpochWithPercentagePrinter(totalEpoch = EPOCHS),
    BatchWithPercentagePrinter(),
    AccuracyPrinter(),
    LossPrinter(lossFunc = lossFunc)
]

# 2. 이렇게 담은 배열을 사용하여 전체 Logger 클래스를 정의합니다.
logger = HistoryLogger(printerList = loggers, isTrain=True)

In [342]:
logger # __repr__ 때문에 수정된 화면을 확인할 수 있습니다.

HistoryPrinter()

In [343]:
# 담긴 Printer의 리스트는 allStackedPrinter()로 출력합니다.
logger.allStackedPrinter()

Printer #0 : EpochWithPercentagePrinter()
Printer #1 : BatchWithPercentagePrinter()
Printer #2 : AccuracyPrinter()
Printer #3 : LossPrinter(lossFunc = CrossEntropyLoss())


In [344]:
# 3. 로그는 다음과 같이 사용합니다.
logger.show(nowEpoch = 1, output = torch.Tensor([[0.9,0.1]]), label = torch.Tensor([1]).long(), index = 1, totalLen = 1000)

Epoch : [1 / 3](33.3%)	Proceeding.. [1 / 1000](0.100%)	Accuracy : [0.000%]	Loss : [1.171101]


이 클래스를 실제 학습시에 사용하면 다음과 같은 형태로 사용합니다.

In [345]:
# 학습용 함수 생성
def train(model, train_loader, optimizer, log_interval):
    model.train()
    
    for batch_idx, (image,label) in enumerate(train_loader):
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        
        optimizer.zero_grad()
        output = model(image)
        
        loss = lossFunc(output, label)
        loss.backward()
        optimizer.step()
        
        if batch_idx % log_interval == 0: # log_interval 주기마다 실시합니다.
            
            # 처음 설정할 때 연결한 모듈에서 필요한 모든 파라미터를 넘겨줍니다.
            logger.show(nowEpoch = Epoch, output = output, label = label, index = batch_idx * len(image), totalLen = len(train_loader.dataset))
            
            

In [346]:
# Evaluation 용 Logger는 다음과 같이 만듭니다.
loggers = [
    EpochPrinter(totalEpoch = EPOCHS),
    AccuracyPrinter(),   
]

# isTrain이 False인 경우 평가용 텍스트가 따로 출력됩니다.
testLogger = HistoryLogger(printerList = loggers, isTrain=False)
testLogger

HistoryPrinter()

In [347]:
# 평가용 메소드
def evaluate(model, test_loader):
    model.eval()
    
    with torch.no_grad():
        for image, label in test_loader:
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            output = model(image)
            
        testLogger.show(nowEpoch = Epoch, output = output, label = label)

In [348]:
# 학습 진행
for Epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, log_interval=200)
    evaluate(model, test_loader)

Epoch : [1 / 3](33.3%)	Proceeding.. [0 / 60000](0.000%)	Accuracy : [3.125%]	Loss : [2.395083]
Epoch : [1 / 3](33.3%)	Proceeding.. [6400 / 60000](10.667%)	Accuracy : [18.750%]	Loss : [2.302021]
Epoch : [1 / 3](33.3%)	Proceeding.. [12800 / 60000](21.333%)	Accuracy : [6.250%]	Loss : [2.306310]
Epoch : [1 / 3](33.3%)	Proceeding.. [19200 / 60000](32.000%)	Accuracy : [3.125%]	Loss : [2.312098]
Epoch : [1 / 3](33.3%)	Proceeding.. [25600 / 60000](42.667%)	Accuracy : [15.625%]	Loss : [2.281388]
Epoch : [1 / 3](33.3%)	Proceeding.. [32000 / 60000](53.333%)	Accuracy : [6.250%]	Loss : [2.330054]
Epoch : [1 / 3](33.3%)	Proceeding.. [38400 / 60000](64.000%)	Accuracy : [18.750%]	Loss : [2.310244]
Epoch : [1 / 3](33.3%)	Proceeding.. [44800 / 60000](74.667%)	Accuracy : [12.500%]	Loss : [2.249013]
Epoch : [1 / 3](33.3%)	Proceeding.. [51200 / 60000](85.333%)	Accuracy : [21.875%]	Loss : [2.330101]
Epoch : [1 / 3](33.3%)	Proceeding.. [57600 / 60000](96.000%)	Accuracy : [9.375%]	Loss : [2.290149]
Epoch 1 Tes

# 왜 이 클래스가 SOLID 원칙을 만족하는가

이 프로젝트(모듈)은 SOLID 원칙을 준수하며 설계하려고 노력했습니다. 왜 이를 만족하는지 SOLID의 다섯 가지 원칙을 살펴보며 확인해 보려고 합니다.

1. Single Responsibility Principle(단일 책임 원칙)

단일 책임 원칙이란, 하나의 클래스는 하나의 기능만을 가지는 원칙입니다.
제 프로젝트의 클래스들은 각각이 출력하는 내용으로 명확히 구분되어 있습니다.(예를 들어 `Accuracy` 를 출력하는 클래스와 `Loss` 를 출력하는 클래스가 따로 있습니다.
또한 `HistoryLogger` 클래스는 비슷한 클래스가 없어서 독립적인 책임을 갖는다고 할 수 있습니다.

2. Open-Closed Principle(개방-폐쇄 원칙)

개방-폐쇄 원칙이란 기존의 코드를 변경하지 않고 기능을 수정하거나 추가할 수 있도록 설계해야 한다는 원칙입니다.

제 프로젝트의 Printer 클래스들은, 내용 수정이 필요할 시 해당 클래스의 `show()` 메소드만 수정하면 됩니다. 이를 위한 다른 기능들은 수정할 필요가 없습니다. 따라서 개방-폐쇄 원칙을 만족한다고 할 수 있습니다.

3. LisKov Substitution Principle(리스코프 치환 법칙)
리스코프 치환 원칙은 부모 클래스에서 가능한 행위를 수행할 수 있어야 한다는 원칙입니다.

제 프로젝트의 자식 클래스들은 상속받은 부모 클래스의 메소드들을 모두 오류 없이 실행 가능하므로 이를 만족한다고 할 수 있습니다.

4. Dependency Inversion Principle(의존 역전 원칙)

의존관계를 맺을 때 구체적인 클래스보다 추상적인 클래스와 관계를 맺는다는 원칙입니다.

제 프로젝트의 Printer 클래스들은 모두 기본이 되는 `BasicPrinter` 추상 클래스를 상속받아 실행되기 때문에 이를 만족한다고 할 수 있습니다.

5. 